In [3]:
df

,timestamp,level,context,message
0,2024-11-15 15:44:30.720,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,Starting WSO2 Carbon...
1,2024-11-15 15:44:30.739,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,"Operating System : Linux 5.4.0-198-generic, amd64"
2,2024-11-15 15:44:30.739,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,Java Home : /opt/java/openjdk
3,2024-11-15 15:44:30.740,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,Java Version : 11.0.10
4,2024-11-15 15:44:30.740,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,Java VM : OpenJDK 64-Bit Server VM 11...
...,...,...,...,...
1051828,2025-01-22 23:59:45.668,WARN,org.wso2.carbon.databridge.agent.endpoint.Data...,"No receiver is reachable at reconnection, will..."
1051829,2025-01-22 23:59:45.674,INFO,org.wso2.carbon.databridge.core.DataBridge,user adminsuper@nosi.cv connected
1051830,2025-01-22 23:59:45.676,ERROR,org.wso2.carbon.databridge.agent.endpoint.Data...,Error while trying to connect to the endpoint....
1051831,2025-01-22 23:59:45.676,ERROR,org.wso2.carbon.databridge.agent.endpoint.Data...,Error while trying to connect to the endpoint....
